#### Study the dataset's meanIR, IRLbl and SCUMBLE metrics
Use the metrics implementet according Charte et al. -- which are used to implement undersampling and oversampling -- to get a better feeling for the label distribution.

In [1]:
import sys
sys.path.append('..')
import json
import help_functions as hf
import numpy as np
hf.setup_gpu(gpu_nr=1)

2 Physical GPUs, 1 Logical GPU


Use the test set for faster computations as its distribution follows the same distribution as the whole data.

In [2]:
with open('../training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(1)]

config['batch_size'] = 32
config['nr_classes'] = 20
test = hf.get_flow(df_file='../' + config['data_folder'] + '/test_df.json.bz2',
                   nr_classes=config['nr_classes'],
                   batch_size=config['batch_size'],
                   image_dimension=config['image_dimension'])
y_true_test = hf.get_y_true(shape=(test.samples, len(test.class_indices)), classes=test.classes)

Found 50000 non-validated image filenames belonging to 40 classes.


../help_functions.py:285: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 49514 validated image filenames belonging to 20 classes.


Imbalance ratio per label (IRLbl)

In [3]:
IRLbl = hf.imbalance_ratio_per_label(y_true_test)
IRLbl

array([24.49017199, 13.29886591, 46.90588235,  1.59966298, 10.53646934,
        6.54679803,  2.28717302, 61.15030675, 18.63084112, 46.46853147,
        2.51038912,  1.91388249,  1.90255774,  1.        , 28.5193133 ,
        7.90130797, 44.3       ,  1.13719338,  8.62613587,  8.19358816])

Mean imbalance ratio (meanIR)

In [4]:
meanIR = hf.mean_imbalance_ratio(y_true_test, class_names=test.class_indices)
print(f'Mean imbalance ratio (meanIR): {meanIR}')

{'Architecture': 24.49, 'Art': 13.3, 'Biology': 46.91, 'Culture': 1.6, 'Entertainment': 10.54, 'Events': 6.55, 'History': 2.29, 'Landscapes': 61.15, 'Language': 18.63, 'Music': 46.47, 'Nature': 2.51, 'Objects': 1.91, 'People': 1.9, 'Places': 1.0, 'Plants': 28.52, 'Politics': 7.9, 'Science': 44.3, 'Society': 1.14, 'Sports': 8.63, 'Technology': 8.19}
Mean imbalance ratio (meanIR): 16.89595354970558


SCUMBLE (Score of ConcUrrence among iMBalanced LabEls)

In [5]:
scumble_dataset, scumble_samples = hf.scumble(y_true_test)
print(scumble_dataset)

0.4355578321363583


Study the labels' entangleness/coherence (TODO: better visualization)

In [7]:
test.class_indices

{'Architecture': 0,
 'Art': 1,
 'Biology': 2,
 'Culture': 3,
 'Entertainment': 4,
 'Events': 5,
 'History': 6,
 'Landscapes': 7,
 'Language': 8,
 'Music': 9,
 'Nature': 10,
 'Objects': 11,
 'People': 12,
 'Places': 13,
 'Plants': 14,
 'Politics': 15,
 'Science': 16,
 'Society': 17,
 'Sports': 18,
 'Technology': 19}

In [9]:
from collections import Counter
np.array(test.classes).sort()
asd = np.array([np.sort(np.array(el)) for el in np.array(test.classes)])
asd_tuple = [tuple(row) for row in asd]
Counter(asd_tuple).most_common(25)

[((13,), 6675),
 ((17,), 2342),
 ((10,), 2134),
 ((3,), 1943),
 ((12,), 1515),
 ((6,), 1299),
 ((13, 17), 1267),
 ((11,), 1243),
 ((11, 13), 1184),
 ((10, 13), 896),
 ((12, 17), 863),
 ((3, 13), 742),
 ((3, 17), 652),
 ((6, 13), 614),
 ((3, 12, 17), 581),
 ((10, 17), 553),
 ((3, 12), 488),
 ((11, 17), 481),
 ((5,), 434),
 ((10, 13, 17), 415),
 ((6, 11, 13), 413),
 ((11, 13, 17), 374),
 ((3, 11), 358),
 ((6, 11), 348),
 ((6, 17), 338)]